In [0]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from itertools import count
from IPython.display import clear_output
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from scipy.stats import pearsonr
from tqdm import tqdm

In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 32
NUM_CLASSES = 10
MODEL_ADDITION_DELTA = 0.01
MODEL_ADDITION_PATIENCE = 3
NR_OF_RUNS = 10
MODEL_NAME = "CIFAR_10_bagging_simpel"
PATH = ""
VOTING = "SOFT"

# Preprocess

In [11]:
(x_train_val, y_train_val), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [0]:
def CIFARmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Flatten()(inputs)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

# Predict

In [0]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y):
    
    if VOTING == "SOFT":
      prediction = soft_voting(models, X)
    elif VOTING == "HARD":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")

    return accuracy_score(prediction, np.argmax(Y, axis=1))

# Train

In [17]:
for run in range(1, NR_OF_RUNS+1):

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    models = []
    accuracies = [0]
    patience = 0

    for i in count(1):

        print(f"\n ===== Train model {i} =====")

        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)

        # Bootstrapping
        idx = np.random.choice(len(x_train), size=len(x_train), replace=True)

        x_train_model = x_train[idx]
        y_train_model = y_train[idx]

        # Create the model
        model = CIFARmodel(IMAGE_SIZE, NUM_CLASSES, 3)
        
        # Load the weighs if the model is already trained
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{i}.h5"

        if os.path.exists(weights_path):
            print(f"Skipping training of model {i}: weights exists")
            model.load_weights(weights_path)
        else:
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{i}.csv", separator=';')

            model.fit(x_train_model,y_train_model,
                      batch_size = BATCH_SIZE,
                      epochs = EPOCHS,
                      validation_data = (x_val, y_val),
                      shuffle = True,
                      callbacks=[es, csv_logger])
            
            model.save_weights(weights_path)
        
        models.append(model)

        acc = predict(models, x_val, y_val)
        delta = acc - accuracies[-1]

        accuracies.append(acc)

        if delta >= MODEL_ADDITION_DELTA:
          patience = 0
        else:
          patience += 1

        print(f"Model: {i} added. Resulting score: {acc}, Delta: {delta}, Patience: {patience}")

        if patience >= MODEL_ADDITION_PATIENCE:
          break

    # Results

    ## Accuracy vs nr of models
    ## Visualizing the accuracy vs the number of models in the ensamble

    print("\n ===== Accuracy vs nr of models =====")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy_{VOTING}.csv")
    display(accuracy_df)

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(models, x_test, y_testc)
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("\n ===== Correlation =====")
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in predictions:
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix_{VOTING}.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + f"/results_{VOTING}.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)

    clear_output(wait=True)


 ===== Train model 1 =====
Train on 40000 samples, validate on 10000 samples
Epoch 1/9999
40000/40000 [==============================] - 2s 40us/step - loss: 2.0426 - accuracy: 0.2722 - val_loss: 1.9313 - val_accuracy: 0.3242
Epoch 2/9999
40000/40000 [==============================] - 2s 39us/step - loss: 1.8545 - accuracy: 0.3503 - val_loss: 1.8269 - val_accuracy: 0.3610
Epoch 3/9999
40000/40000 [==============================] - 2s 39us/step - loss: 1.7863 - accuracy: 0.3767 - val_loss: 1.7762 - val_accuracy: 0.3808
Epoch 4/9999
40000/40000 [==============================] - 2s 38us/step - loss: 1.7459 - accuracy: 0.3917 - val_loss: 1.7403 - val_accuracy: 0.3996
Epoch 5/9999
40000/40000 [==============================] - 2s 38us/step - loss: 1.7159 - accuracy: 0.4029 - val_loss: 1.7141 - val_accuracy: 0.4140
Epoch 6/9999
40000/40000 [==============================] - 2s 38us/step - loss: 1.6920 - accuracy: 0.4121 - val_loss: 1.7055 - val_accuracy: 0.4117
Epoch 7/9999
40000/40000 [==

KeyboardInterrupt: ignored